In [1]:
import torch 
import numpy as np
import pandas  as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels= 1, out_channels= 6 , kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels= 6, out_channels= 12 , kernel_size=5)
        
        self.fc1 = nn.Linear(in_features= 12*4*4 , out_features= 120)
        self.fc2 = nn.Linear(in_features= 120, out_features= 60)
        self.out = nn.Linear(in_features= 60, out_features= 10)

    def forward(self,t):
        
        #1 inpput layer 
        
        t = t
        
        #2 hidden conv layer
        
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size = 2 , stride = 2)
        
        #3 hidden conv layer 
        
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size = 2 , stride = 2)
        
        #4 Linear layer 
        
        t = t.reshape(-1, 12*4*4)     #flattening is hapening here
        t = self.fc1(t)
        t = F.relu(t)

        
        #5 Linear layer 
        
        t = self.fc2(t)
        t = F.relu(t)
        
        #6 output layer
        
        t = self.out(t)
        # t = F.softmax(t,dim = 0)    but this line is not required because we will predict the output later and softmax will be used explicitly later
        return t
        
#loading data from url of Mnist

train_set = torchvision.datasets.FashionMNIST(
        root = './data'
        ,train = True
        ,download=True
        ,transform = transforms.Compose([
            transforms.ToTensor()
        ])
    
)

        
data_loader = torch.utils.data.DataLoader(train_set,
                    batch_size= 100                  
                    )


In [10]:
from itertools import product
parameters = dict(
             batch_size = [1000,5000,10000],
lr = [.001,.0001],
shuffle = [True,False]
)
param_values = [v for v in parameters.values()]
param_values

[[1000, 5000, 10000], [0.001, 0.0001], [True, False]]

In [13]:
def get_num_correct(preds,labels):
    
    return preds.argmax(dim = 1).eq(labels).sum().item()



In [29]:
for batch_size,lr,shuffle in product(*param_values):
    print(batch_size,lr,shuffle)

1000 0.001 True
1000 0.001 False
1000 0.0001 True
1000 0.0001 False
5000 0.001 True
5000 0.001 False
5000 0.0001 True
5000 0.0001 False
10000 0.001 True
10000 0.001 False
10000 0.0001 True
10000 0.0001 False


# Another Aproach is to use Runbuilder


In [3]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

In [4]:
params = OrderedDict(
            
            lr = [.01,.001],
            batch_size = [100,1000,10000],
            shuffle = [True,False]
)

In [5]:
#building the class

class RunBuilder():
    @staticmethod
    
    def get_runs(params):
        
        
        Run = namedtuple('Run',params.keys())
        
        runs = []
        
        for v in product(*params.values()):
            runs.append(Run(*v))
        return runs
        

In [54]:
params1 = OrderedDict(
            
            batch_size = [100,1000],
            lr = [.01,.001],
            device = ['cuda', 'cpu']
) 
Runs = RunBuilder.get_runs(params1)
Runs = RunBuilder.get_runs(params)
Runs

[Run(lr=0.01, batch_size=100, shuffle=True),
 Run(lr=0.01, batch_size=100, shuffle=False),
 Run(lr=0.01, batch_size=1000, shuffle=True),
 Run(lr=0.01, batch_size=1000, shuffle=False),
 Run(lr=0.01, batch_size=10000, shuffle=True),
 Run(lr=0.01, batch_size=10000, shuffle=False),
 Run(lr=0.001, batch_size=100, shuffle=True),
 Run(lr=0.001, batch_size=100, shuffle=False),
 Run(lr=0.001, batch_size=1000, shuffle=True),
 Run(lr=0.001, batch_size=1000, shuffle=False),
 Run(lr=0.001, batch_size=10000, shuffle=True),
 Run(lr=0.001, batch_size=10000, shuffle=False)]

In [14]:
test_net = Network()
epochs = 6
for batch_size,lr,shuffle in product(*param_values):
    print(batch_size,lr,shuffle)
    comment = f'batch size = {batch_size},lr = {lr},shuffle = {shuffle}'
    
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size ,shuffle=shuffle)
    images,labels = next(iter(train_loader))
    tb = SummaryWriter(comment = comment)
    grid = torchvision.utils.make_grid(images)
    

    tb = SummaryWriter(comment = comment)
    tb.add_image('images',grid)
    tb.add_graph(test_net,images)

    optimizer = optim.Adam(test_net.parameters(), lr=lr)


    for epoch in range(epochs):

        total_loss = 0
        total_correct = 0
        for batch in train_loader:                      # Get Batch
            images, labels = batch

            preds = test_net(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad()
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            total_loss += loss.item()*images.shape[0]               #using it to generalise the loss for different batch sizes
            total_correct += get_num_correct(preds,labels)

        tb.add_scalar("loss",total_loss,epoch)
        tb.add_scalar("total correct", total_correct,epoch)
        tb.add_scalar("Accuracy", total_correct/len(train_set), epoch)

        tb.add_histogram("conv1 bias", test_net.conv1.bias, epoch)
        tb.add_histogram("conv1.weight", test_net.conv1.weight,epoch)
        tb.add_histogram("conv1.weight.grad", test_net.conv1.weight.grad,epoch)

        print("epoch:", epoch  , "total loss:", total_loss, "total correct:" ,total_correct )

tb.close()


1000 0.001 True
epoch: 0 total loss: 97761.58267259598 total correct: 25695
epoch: 1 total loss: 47783.6109995842 total correct: 42002
epoch: 2 total loss: 41220.34651041031 total correct: 44318
epoch: 3 total loss: 37806.72359466553 total correct: 45578
epoch: 4 total loss: 35221.113204956055 total correct: 46653
epoch: 5 total loss: 33312.54795193672 total correct: 47339
1000 0.001 False
epoch: 0 total loss: 33592.165023088455 total correct: 47358
epoch: 1 total loss: 31433.259904384613 total correct: 48341
epoch: 2 total loss: 30614.614009857178 total correct: 48764
epoch: 3 total loss: 29818.369656801224 total correct: 49154
epoch: 4 total loss: 29040.54895043373 total correct: 49481
epoch: 5 total loss: 28308.53235721588 total correct: 49784
1000 0.0001 True
epoch: 0 total loss: 27340.08577466011 total correct: 50247
epoch: 1 total loss: 27148.66927266121 total correct: 50338
epoch: 2 total loss: 27016.471296548843 total correct: 50361
epoch: 3 total loss: 26947.718918323517 total